# Gatys等人文章代码的复现

## 导入必要的包

In [22]:
# 首先一定要导入的是torch 
import torch
# 需要读入图像, 所以需要PIL
from PIL import Image
# 由于需要做图像处理, 所以应该导入Transforms
from torchvision import transforms
# 由于需要搭建神经网络, 所以应该导入torch.nn
from torch import nn

from torchvision import models as vision_models

In [23]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"now is using {device}")

now is using cuda


## 数据准备

1. 构建dataset与dataloader的环节, 但是由于gatys等人的工作不需要这么做, 所以不应该有这两个东西
    1. 取而代之的是三张图像：风格图、内容图与风格化的噪声图

完成:20240306

In [24]:
# 将风格图、内容图引入内存
content_img_pil = Image.open('./data/VGG19/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')
style_img_pil = Image.open('./data/VGG19/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg')

# 将风格图、内容图转换为Tensor类型, 并创建噪声图
totensor = transforms.ToTensor()

content_img = totensor(content_img_pil).to(device=device) # 内容图
style_img = totensor(style_img_pil).to(device=device) # 风格图
styled_img = content_img # 风格化图像


print(content_img.device)
print(style_img.device)
print(styled_img.device)

cuda:0
cuda:0
cuda:0


2. 需要构建神经网络
    1. 这一步需要看gatys等人的原文, 知道需要哪些具体的部件
       1. 使用19层vgg网络中的16个卷积层与5个池化层
          1. 通过缩放权重来标准化(Normalize)网络，使得图像和位置上的每个卷积滤波器的平均激活等于 1。
       2. 不使用任何全连接层
       3. 使用平均池化代替最大池化
       4. Content Representation使用了19层VGG网络中的'conv4_2'
       5. Style Representation使用了19层VGG网络中的'conv1_1', 'conv2_1', 'conv3_1', 'conv_4_1', 'conv_5_1'
    2. 根据需要的部件对输入的数据进行处理
    3. 创建神经网络的实例

以下是构建神经网络的过程

由于需要使用预训练的VGG19网络, 所以我们先下载该网络

In [25]:
# 下载VGG19模型到本地
pretrained_vgg19 = vision_models.vgg19(weights=vision_models.VGG19_Weights.IMAGENET1K_V1)
    # vision_models.vgg.VGG19_Weights与vision_models.VGG19_Weights是同一个类
    # 如果下载慢, 可以在命令行中运行这条命令

# path = './data/VGG19/VGG19_pretrained.pth'
# torch.save(pretrained_vgg19, path)
print(type(pretrained_vgg19))
vision_models.vgg.vgg19

<class 'torchvision.models.vgg.VGG'>


<function torchvision.models.vgg.vgg19(*, weights: Union[torchvision.models.vgg.VGG19_Weights, NoneType] = None, progress: bool = True, **kwargs: Any) -> torchvision.models.vgg.VGG>

In [29]:
# print(pretrained_vgg19.state_dict)

dict = pretrained_vgg19.state_dict()
# print(dict)

i = 0
# for key, value in dict.items():
#     i += 1
#     print(f"the {i}th of the dict, key is {key}, the type of key is {type(key)}, value is {type(value)}")

# print(pretrained_vgg19.features[0].weight==dict['features.0.weight'])
# print(pretrained_vgg19.features[0], type(pretrained_vgg19.features[0]))
conv_1 = pretrained_vgg19.features[0]

# print(type(conv_1))
# dir(conv_1)
# print(conv_1.weight)
# # dir(torch.nn.modules.conv.Conv2d)


In [41]:
# print(dict['features.0.weight'])
print(dict['Conv_0'])

KeyError: 'Conv_0'

In [1]:
class GatysNet(nn.Module):
    def __init__(self, vgg19):
        super().__init__(self, vgg19)
        self.style_layer1 = pretrained_vgg19.

SyntaxError: unexpected EOF while parsing (2037666928.py, line 1)

3. 定义学习率与epoch

## 训练与测试过程的定义 

- 训练过程主要用于计算损失函数, 并根据损失函数与梯度更新参数
  1. gatys等人的损失函数十分明确, 分为两个部分, 明天可以根据论文编写对应的代码

损失函数中参数的定义如下：


### Content Representation


1.  基础参数定义
    1.  $N_l $：网络第 $l$ 层中卷积核的数量
    2.  $M_l$：图像的在网络第 $l$ 层中的卷积核的处理下, 输出的某张特征图的长与宽之积, 满足 $M_l=width\times length$
    3.  $F^l$：图像在网络第$l$层中所有$N^l$个卷积核的处理下, 输出的特征图组, $F^l\in R^{N_l\times M_l}$
    4.  $F_{ij}^l$ ：图像在网络第$l$层的第$i$个卷积核的处理下, 输出的特征图中的第$j$个像素
    5.  $\vec p$：输入的原图像
    6.  $\vec x$：生成的图像
    7.  $\vec P^l$：输入图像 $\vec p$ 在第$l$层上的特征图, 有 $\vec P^l \in F^l \in R^{N_l\times M_l}$
    8.  $\vec F^l$：生成图像 $\vec x$ 在第 $l$ 层上的特征图, 有 $\vec F^l \in F^l \in R^{N_l\times M_l}$
2.  内容损失函数定义
    1.  $$\mathcal{L}_\text{content}(\vec p, \vec x, l)=\frac 1 2 \sum_{ij}(F_{ij}^l-P_{ij}^l)^2$$


###  Style Representation

####  Gram矩阵


1.  参数设定
    1.  $\vec F^l_i$：是一个向量, 表示图像在网络第 $l$ 层、第 $i$ 个卷积核的输出结果, 是一个 $1\times M_l$的向量
        1.  在网络的第 $l$ 层中, 一共有 $N_l$这样的向量
    2.  $F^l_{ik}$：是一个标量, 是向量 $\vec F^l_i$ 的一个分量, 表示图像在网络第 $l$ 层、第 $i$ 个卷积核的输出结果中的第 $k$个分量
2.  Gram矩阵 $G^l \in R^{N_l \times N_l}$
    1.  $$\begin{aligned}G^l &= \begin{bmatrix}\vec F^l_1, \vec F^l_2, \cdots, \vec F^l_i, \vec F^l_{N_l}\end{bmatrix} \cdot \begin{bmatrix}\vec F^l_1\\ \vec F^l_2\\ \vdots\\ \vec F^l_i\\ \vec F^l_{N_l}\end{bmatrix}\\ &= \begin{bmatrix} (\vec F^l_1,\vec F^l_1), (\vec F^l_1,\vec F^l_2),\cdots, (\vec F^l_1,\vec F^l_{N_l})\\  (\vec F^l_2,\vec F^l_1), (\vec F^l_2,\vec F^l_2),\cdots, (\vec F^l_2,\vec F^l_{N_l})\\ \vdots, \vdots,\vdots, \vdots\\ (\vec F^l_{N_l},\vec F^l_1), (\vec F^l_{N_l},\vec F^l_2),\cdots, (\vec F^l_{N_l},\vec F^l_{N_l})\end{bmatrix}\end{aligned}$$
    2.  在具体实现方面, 直接将网络第 $l$ 层输出的 $N_l$个特征图逐一相乘即可得到Gram矩阵 $G^l \in R^{N_l \times N_l}$
    3.  若想获取Gram矩阵 $G^l$ 中的第 $(i, j)$ 元素 $G^l_{ij}$, 直接用网络第 $l$ 层的第 $i$ 个输出乘以 第 $j$ 个输出即可, 即 $\vec F^l_i\times \vec F^l_j$

#### 风格损失

网络第 $l$ 层对风格损失的贡献 $E_l$ 如下：

1.  参数设定
    1.  $\vec a$：原始风格图像
    2.  $\vec x$：生成图像
    3.  $A^l$：原始图像 $\vec a$ 在第$l$层上的Gram矩阵
    4.  $G^l$：生成图像 $\vec x$ 在第$l$层上的Gram矩阵
    5.  $L$：神经网络的层数(含有卷积核的层数)
2.  第 $l$ 层对风格损失的贡献 $E_l$ 的计算公式：
    1.  $$E_l = \frac{1}{4N_l^2 M_l^2} \sum_{i,j} \left( A_{ij}^l -G_{ij}^l \right)^2$$
        1.  $A_{ij}^l$ 的计算：将原始风格图像$\vec x$输入网络, 用网络第 $l$ 层的第 $i$ 个输出乘以 第 $j$ 个输出即可
        2.  $G_{ij}^l$ 的计算：将生成图像$\vec x$输入网络, 用网络第 $l$ 层的第 $i$ 个输出乘以 第 $j$ 个输出即可
3.  风格损失函数的公式
    1.  $$\mathcal{L}_\text{style}(\vec a, \vec x) = \sum_{l=0}^L w_lE_l$$
        1.  $w_l$：权重参数
        2.  wl = 1/5 in those layers, wl = 0 in all other layers 


### 总损失函数

总损失函数的公式如下所示

$$\mathcal{L}_\text{total}(\vec p,\vec a, \vec x) = \alpha \mathcal{L}_\text{content}(\vec p, \vec x)+\beta \mathcal{L}_\text{style}(\vec a, \vec x)$$


The ratio α/β was either 1 × 10−3 (Fig 3 B), 8 × 10−4 (Fig 3 C), 5 × 10−3 (Fig 3 D), or 5 × 10−4 (Fig 3 E, F).


  2. 测试过程主要用于告知电脑面前的我们, 当前训练到了什么程度, 这里主要写的是评价指标, 当然也可以直接打印当前损失函数的值

## 优化过程的创立

1. 编写损失函数
2. 创建优化器

## 训练过程

使用训练调用train_loop与test_loop